# HYBRID

<br><br>
We use Content-Based --> Colaborative to build recomender system. The result of recomendation by Content-Based, will be further analyze and try to predict its rating using Colaborative-Filter. Rating that is more than 3 will be recomended to users. 
<br><br>
For Content-Based, we use Cosine SImilarity as it is more accurate to detect the similarity between users. (You can see its defenition in [this link](https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243)). We use data IMDB top 250 and Data from our bootcamp's class survey.
<br><br>
Our Thought process is in general define by :<br><b>
1. Input your name (of course it should be in data)
2. Use Content based to show which film is recomendeded to your name
3. Use Collaborative to predict the rating of those recomended movies. 
4. Predicted ratings that is more than 3.7 (abritary numbers, can customize) will be recomended to user
<br><br>
</b>
Noted that, in our data, we use RatingUser.csv that has only 10 movies that has been rated by users. Also see our previous Colaborative and Content Based for more explanation. Here, we fast forward. We use CountVectorizer and cosine similarity for Content Based and Use Cosine with KNN

### Data Preprocessing

In [14]:
import pandas as pd
import numpy as np

# For Content Based
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import CountVectorizer

# For Collaborative Filtering
from surprise import Dataset
from surprise import Reader

from surprise import KNNBasic


<br><br>
We read our data. There are 2 data, <i>Movie Genre.csv</i> for Content Based and <i>RatingUser.csv</i> for Colaborative. Because there are 250 movies in <i>Movie Genre.csv</i> and only 10 in <i>RatingUser.csv</i>, we only acquired those 10 films in our data.<br>

In [2]:
data1 = pd.read_csv('RatingUser.csv',delimiter=',')
data2 = pd.read_csv('Movie Genre.csv',delimiter=';')

data3=pd.DataFrame(columns=list(data2.columns))
for i in range(data2.shape[0]):
    if data2.iloc[i,1] in list(data1.columns):
        dummy=[data2.iloc[i,:]]
        data3=data3.append(dummy,ignore_index=True)
data3['Title'].unique

<bound method Series.unique of 0                         Gladiator
1                            Wall-e
2            Spider-Man: Homecoming
3                  The Wizard of Oz
4                    The Terminator
5                              Jaws
6                          Zootopia
7    Guardians of the Galaxy Vol. 2
8                      The Avengers
9                           Aladdin
Name: Title, dtype: object>

## Content-Based Recomendation System

<br><br>
First, Input your name in below cell. You can see which movie you already watched.
<br>

In [3]:
Name='Mulya'
print("\nInput names : ",Name)

df = (data1.set_index(['Nama Anda']).stack().reset_index(name='a'))
df.columns=['Nama Anda','Movie','Rating']
movie_watched = np.array(df[df['Nama Anda']==Name]['Movie'])
print('\nMovie already watched : ',movie_watched)


Input names :  Mulya

Movie already watched :  ['The Avengers' 'Zootopia' 'Aladdin' 'Spider-Man: Homecoming'
 'Guardians of the Galaxy Vol. 2']


<br><br>
We only used Rated,Genre,Language,Country,Plot,Director,Actors to determine which movies these input name likes. Then we merge those columns into one column join.

In [4]:
data3=data3[['Title','Rated','Genre','Language','Country','Plot','Director','Actors']]
data3=data3.set_index('Title')
del data3.index.name
data3.head()

,Rated,Genre,Language,Country,Plot,Director,Actors
Gladiator,R,"Action, Adventure, Drama",English,"USA, UK",When a Roman general is betrayed and his famil...,Ridley Scott,"Russell Crowe, Joaquin Phoenix, Connie Nielsen..."
Wall-e,G,"Animation, Adventure, Family",English,USA,"In the distant future, a small waste-collectin...",Andrew Stanton,"Ben Burtt, Elissa Knight, Jeff Garlin, Fred Wi..."
Spider-Man: Homecoming,PG-13,"Action, Adventure, Sci-Fi",English,USA,"Peter Parker, with the help of his mentor Tony...",Jon Watts,"Tom Holland, Michael Keaton, Robert Downey Jr...."
The Wizard of Oz,G,"Adventure, Family, Fantasy",English,USA,Dorothy Gale is swept away from a farm in Kans...,"Victor Fleming, George Cukor, Mervyn LeRoy, No...","Judy Garland, Frank Morgan, Ray Bolger, Bert Lahr"
The Terminator,R,"Action, Sci-Fi","English, Spanish","UK, USA",A seemingly indestructible humanoid cyborg is ...,James Cameron,"Arnold Schwarzenegger, Michael Biehn, Linda Ha..."


<br><br>
We see that in Actors, Genre, there are comma. We need to remove those commas using Regex

In [5]:

data3['Join']=data3.apply(lambda x: ' '.join(x), axis = 1)

data3=data3.replace(regex=True,to_replace=r',',value=r'')['Join']
data3=pd.DataFrame(data3)
data3.head()

,Join
Gladiator,R Action Adventure Drama English USA UK When a...
Wall-e,G Animation Adventure Family English USA In th...
Spider-Man: Homecoming,PG-13 Action Adventure Sci-Fi English USA Pete...
The Wizard of Oz,G Adventure Family Fantasy English USA Dorothy...
The Terminator,R Action Sci-Fi English Spanish UK USA A seemi...


<br><br>
Now, we use CountVectorizer to count and label those columns

In [10]:
count = CountVectorizer()
count_matrix = count.fit_transform(data3['Join'])

# generating the cosine similarity matrix
cos_sim = cosine_similarity(count_matrix, count_matrix)
ind=list(data3.index)
cor=pd.DataFrame(cos_sim)
cor.index=ind
cor.columns=ind

<br><br><br>
Then, we sort 3 best movies that is highly corelated to our already watched movies

In [11]:
rec=cor[movie_watched].drop(index=movie_watched)
rec2=pd.DataFrame(rec.mean(axis=1),columns=['Score']).sort_values('Score',ascending=False)
recomendation = list(rec2.index[:3])
print('Film Recomendation : \n',recomendation)

Film Recomendation : 
 ['Gladiator', 'The Wizard of Oz', 'The Terminator']


In [12]:
pd.DataFrame(rec.mean(axis=1),columns=['Score']).sort_values('Score',ascending=False)

,Score
Gladiator,0.217172
The Wizard of Oz,0.205308
The Terminator,0.194138
Jaws,0.187981
Wall-e,0.161400


<br><br>
Using Content Based, it is recomended to watch Gladiator, The Wizard of Oz, and The Terminator. Now we predict which movie has the highest ratings

## Collaborative

<br><br>
We use KNNBasic and Cosine to calculate the similarity between users

In [15]:
df.head()

,Nama Anda,Movie,Rating
0,Hania,Wall-e,3.0
1,Hania,The Terminator,5.0
2,Hania,The Wizard of Oz,4.0
3,Hania,Jaws,4.0
4,Hania,Gladiator,4.0


In [16]:
reader = Reader(rating_scale=(1, 5))

# Loads Pandas dataframe
data = Dataset.load_from_df(df, reader)

In [17]:
# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between items
}
algo = KNNBasic(sim_options=sim_options)

In [18]:
trainingSet = data.build_full_trainset()

algo.fit(trainingSet)
Sim=pd.DataFrame(algo.compute_similarities(),index=list(df['Nama Anda'].unique()),columns=list(df['Nama Anda'].unique()))
Sim.index=list(df['Nama Anda'].unique())
Sim[Name][Sim[Name]==0].sort_values()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


Hania            0.0
Romantika        0.0
Indra 1991 SM    0.0
Name: Mulya, dtype: float64

<br><br><br>
Using Cosine, we find 3 users that are not highly corelated. If we see below, we can see that it's because between Mulya and those 3 users, there are no similiarity between movies that already been rated. 

In [19]:
compar=pd.DataFrame()
print(df[df['Nama Anda']=='Romantika'])
print(df[df['Nama Anda']=='Hania'])
print(df[df['Nama Anda']=='Mulya'])

    Nama Anda             Movie  Rating
67  Romantika            Wall-e     5.0
68  Romantika  The Wizard of Oz     5.0
  Nama Anda             Movie  Rating
0     Hania            Wall-e     3.0
1     Hania    The Terminator     5.0
2     Hania  The Wizard of Oz     4.0
3     Hania              Jaws     4.0
4     Hania         Gladiator     4.0
   Nama Anda                           Movie  Rating
28     Mulya                    The Avengers     4.0
29     Mulya                        Zootopia     2.0
30     Mulya                         Aladdin     5.0
31     Mulya          Spider-Man: Homecoming     4.0
32     Mulya  Guardians of the Galaxy Vol. 2     3.0


<br><br>
Now, we predict the ratings of recomended movies by Content Based

In [21]:
Movie=df['Movie'].unique()
rec2=[]
for i in Movie:
    if i not in movie_watched:
        prediction = algo.predict(Name, i)
        rec2.append([i,Name,float(prediction.est)])
rec2=pd.DataFrame(rec2)
rec2.columns=['Movie','User','Predicted Rating']


In [22]:
pred_rat=pd.DataFrame(columns=list(rec2.columns))
for i in range(rec2.shape[0]):
    if rec2.iloc[i,0] in recomendation:
        pred_rat=pred_rat.append([rec2.iloc[i,:]])
pred_rat

,Movie,User,Predicted Rating
1,The Terminator,Mulya,3.727418
2,The Wizard of Oz,Mulya,3.601987
4,Gladiator,Mulya,3.729781


<br><br><br>
We get the predicted result above, it's quite similiar. Now we get the highest predicted ratings to be recomended to users

In [27]:
rec_user=pred_rat[pred_rat['Predicted Rating']>=3.7]
print('User {} is recomended to watch {}'.format(Name,rec_user['Movie'].tolist()))

User Mulya is recomended to watch ['The Terminator', 'Gladiator']


### Conclusion

Using Hybrid of Content Based and Colaborative, we got the recomendation movies for user, that is the movies that is similiar of content and has predicted ratings more than 3.7 . 